In [1]:
!cat ../environment.yml

name: maricopa
channels:
- conda-forge
- defaults
dependencies:
- python=2.7
- matplotlib
- jupyter
- pandas
- xarray
- numpy
- mechanize



In [9]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from mechanize import Browser

out_path = '../NetCDF/'
tr = [y+'-01-01' for y in [str(y) for y in range(1980, 2017)]]
tr.append(str(pd.datetime.utcnow().date()))

meta = pd.read_csv("../sensor_ids.csv")
meta['DEV_ID'] = meta['DEV_ID'].astype(int).astype(str)

In [10]:
types = meta.DEV_TYPE.unique()
types

array(['Rptr. Batt.', 'Precip.', 'Stream PT', 'Non Subm PT', 'Dewpoint',
       'Humidity', 'Temperature', 'Bubbler', 'Peak Wind', 'Wind Spd/Dir',
       'Wind Dir.', 'Solar Rad.', 'Pressure', 'Status', 'Flasher',
       'Ave. Wind', 'Radar', 'Water Temp.'], dtype=object)

In [11]:
# these are the IDs of the precip gages that were removed in Oct 2016
for ID in ['4650', '4785', '5280', '5490', '5715', '5820', '6680',]:
    meta.loc[meta.DEV_ID==ID, 'DEV_ID'] = '9'+ID

In [12]:
# skip these types of stations
excluded = ['Rptr. Batt.', 'Status', 'Flasher',]

id1s = [f[0:-3] for f in os.listdir(out_path)]
unprocessed = [s for s in meta.index if (meta.loc[s, 'DEV_ID'] not in id1s) and (meta.loc[s, 'DEV_TYPE'] not in excluded)]
meta.iloc[unprocessed].head()

,DEV_ID,DEV_NAME,DEV_TYPE,DEV_DATE,STA_LAT_DMS,STA_LONG_DMS,STA_ELEV,STA_LOC
248,5098,Winters Wash @ Indian School Rd.,Stream PT,7/14/2005 0:00:00,33 29 37.8,112 55 04.0,1105.0,Indian School Rd. 2 miles E of 411th Ave.
310,5265,Coyote Wash,Precip.,11/27/2002 0:00:00,33 41 49.2,112 55 24.9,1805.0,3.5 miles SW of the intrscn. of Aguila and Wic...
661,6853,Gila @ Estrella Pkwy.,Stream PT,3/1/1993 0:00:00,33 23 20.6,112 23 32.9,905.0,Gila River at Estrella Parkway


In [13]:
print(len(unprocessed))
write_to_file = True

3


In [18]:
for j in unprocessed:
    name = meta.loc[j, 'DEV_TYPE']
    id1 = meta.loc[j, 'DEV_ID']

    L = []
    for i in range(len(tr)-1):
        L0 = fill_form(id1, tr[i], tr[i+1])
        while len(L0) == 0:
            L0 = fill_form(id1, tr[i], tr[i+1])
        if len(L0) > 16:
            if len(L) == 0:
                L = L0[0:-2]
            else:
                L.extend(L0[14:-2])
    # this one has a bad value
    if id1 == '6853':
        drop = [l for l in L if '107.3094628000.0' in l]
        print drop
        L.remove(drop[0])

    try:
        units, s = get_data(L, name)
    except:
        if write_to_file:
            with open('list_issues.txt', 'a') as issues:
                issues.write('\n')
                issues.write(name)
                issues.write(id1)
                issues.write(','.join(L))
            continue
        else:
            print(name, id1)
            break

    if (name=='Temperature' or name=='Pressure' or name=='Water Temp.') and s.shape[1] > 1:
        s = s.iloc[:,0].to_frame()
        units = [units[0]]
    elif name == 'Precip.' and s.shape[1] > 1:
        s = s.iloc[:,1].to_frame()
        units = [units[1]]

    # get lat to a float
    lat = to_decimal(*[float(lat) for lat in meta.loc[j,'STA_LAT_DMS'].split()])

    # get lon to a float and we know it is meant to be in the US so make it negative
    lon = -to_decimal(*[float(lon) for lon in meta.loc[j,'STA_LONG_DMS'].split()])

    # get elevation as a float
    elev = float(meta.loc[j,'STA_ELEV'])

    ds = to_ds(s, lat, lon, elev, units, name)
    ds.attrs.update(meta.loc[j].to_dict())
    ds.to_netcdf('{out_path}{id1}.nc'.format(out_path=out_path, id1=id1), format='netCDF4', engine='netcdf4')
    print id1

['04/03/1994 22:04:42  107.3094628000.0\n']


In [17]:
def fill_form(id1, start, end):
    br = Browser()
    br.open("http://alert.fcd.maricopa.gov/showrpts_mc.html")

    for i, f in enumerate(br.forms()):
        if i==0:
            continue
        else:
            br.form =f

    br.form.set_value(id1, name="ID1", type="text")  

    #set start time and date
    br.form.set_value([start[5:7]], name="ms")
    br.form.set_value([start[8:10]], name="ds")
    br.form.set_value([start[0:4]], name="ys")
    br.form.set_value(["00:00:00"], name="hs")

    #set end time and date
    br.form.set_value([end[5:7]], name="ME")
    br.form.set_value([end[8:10]], name="DE")
    br.form.set_value([end[0:4]], name="YE")
    br.form.set_value(["24:00:00"], name="HE")
    response = br.submit()  # submit current form
    L = [l for l in response.readlines()]
    
    return L

def get_data(L, name):
    s = pd.DataFrame([l.split() for l in L[14:-2]])
    cols = [l.strip() for l in L[13].split('  ') if l.strip()]
    cols.extend(range(s.shape[1]-len(cols)))
    s.columns = cols
    
    # data are in mountain time with no daylight savings so convert to UTC
    dt_index = pd.DatetimeIndex(s['Date']+' '+s['Time'])+pd.DateOffset(hours=7)
    s = s.set_index(dt_index).drop(['Date','Time'], axis=1)
    s.index.name='time'
    
    # change the name of the data variable from units to the actual name
    units = list(s.columns)
    
    if name == 'Stream PT' or name=='Non Subm PT' or name=='Bubbler' or name=='Radar':
        if len(units)==1 and units[0]=='feet':
            s.columns = ['stage']
        elif units[0]=='feet':
            s.columns = ['stage', 'discharge']
            units[1]='cfs'
        
    else:
        name = L[7].strip()
        s.columns = [name]*len(s.columns)
    
    # convert to float
    s = s.astype('float')
    s = s.sort_index()
    return(units, s)
    
def to_decimal(degree, minute, second):
    return(degree+(minute/60.)+(second/3600.))

def to_ds(s, lat, lon, elev, units, name):
    
    ds = s.to_xarray()
    ds['lat'] = lat
    ds['lon'] = lon
    ds['elev'] = elev
    ds.set_coords(['time','lat','lon', 'elev'], inplace=True)
    if s.shape[0] < 20:
        chunk=1
    else:
        chunk = s.shape[0]/20
    print(name)
    if name == 'Precip.':
        attrs = [{'standard_name': 'rainfall'}]
    elif name == 'Temperature':
        attrs = [{'standard_name': 'air_temperature'}]
    elif name == 'Dewpoint':
        attrs = [{'standard_name': 'dew_point_temperature'}]
    elif name == 'Stream PT'or name == 'Non Subm PT' or name=='Bubbler' or name=='Radar':
        attrs = [{'standard_name': 'water_elevation'}, 
                 {'standard_name':'discharge'}]
    elif name == 'Humidity':
        attrs = [{'standard_name': 'relative_humidity'}]
    elif (name == 'Ave. Wind' or name == 'Wind Spd/Dir') and units[0] == 'miles per hour':
        attrs = [{'standard_name': 'wind_speed'}]
    elif name == 'Peak Wind':
        attrs = [{'standard_name': 'max_wind_speed'}]
    elif name == 'Wind Dir.':
        attrs = [{'standard_name': 'wind_from_direction'}]
    elif name == 'Solar Rad.':
        attrs = [{'standard_name': 'solar_radiation'}]
    elif name == 'Pressure':
        attrs = [{'standard_name': 'air_pressure'}]
    elif name == 'Water Temp.':
        attrs = [{'standard_name': 'water_temperature'}]
        
    for i, u in enumerate(units):
        if u == 'degrees F' or u == 'degreesF':
            attrs[i].update({'units': 'degF'})
        elif u == 'inches':
            attrs[i].update({'units': 'inch'})
        else:
            attrs[i].update({'units':  u})
        
    for i, v in enumerate(ds.data_vars.keys()):
        ds[v].attrs.update(attrs[i])
        ds[v].encoding.update({'dtype': np.double,'chunksizes':(chunk,),'zlib': True})
    ds.lat.attrs.update({'units': 'degrees_north',
                         'axis': 'Y',
                         'long_name': 'latitude',
                         'standard_name': 'latitude'})
    ds.lat.encoding.update({'dtype': np.double,'chunksizes':(chunk,),'zlib': True})
    ds.lon.attrs.update({'units': 'degrees_east',
                         'axis': 'X',
                         'long_name': 'longitude',
                         'standard_name': 'longitude'})
    ds.lon.encoding.update({'dtype': np.double,'chunksizes':(chunk,),'zlib': True})

    ds.elev.attrs.update({'units': 'feet',
                         'axis': 'Z',
                         'long_name': 'elevation',
                         'standard_name': 'elevation'})
    ds.elev.encoding.update({'dtype': np.double,'chunksizes':(chunk,),'zlib': True})
    ds.time.encoding.update({'units':'seconds since 1970-01-01', 
                             'calendar':'gregorian',
                             'dtype': np.double,'chunksizes':(chunk,),'zlib': True})

    ds.attrs.update({ 'institution': 'Data from Flood Control District of Maricopa County, hosted by Princeton University',
                      'references': 'http://alert.fcd.maricopa.gov/showrpts_mc.html',
                      'featureType': 'timeSeries',
                      'Conventions': 'CF-1.6',
                      'history': 'Created by Princeton University Hydrometeorology Group at {now} '.format(now=pd.datetime.now()),
                      'author': 'jsignell@princeton.edu'})
    return(ds)